In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = "./malaysia_house_price_data_2025.csv"

df = pd.read_csv(file_path)
print(df.head())

## Township

O nome da localidade específica ou conjunto habitacional onde o imóvel está localizado.
Exemplo: "SCIENTEX SUNGAI DUA".

## Area

A área ou bairro dentro do estado em que a propriedade está localizada.
Exemplo: "Tasek Gelugor".

## State

O estado da Malásia onde a propriedade está situada.
Exemplo: "Penang", "Johor", "Perak".

## Tenure

O tipo de posse do imóvel:
Freehold: Propriedade plena e permanente.
Leasehold: Propriedade por um período limitado (geralmente 99 anos).
Esse fator pode influenciar no valor do imóvel.

## Type

Tipo de propriedade:
Terrace House: Casa geminada (compartilha paredes laterais com outras casas).
Cluster House: Casa com formato de agrupamento (quatro casas por lote).
Pode haver combinações, como "Cluster House, Terrace House".

## Median_Price

Preço mediano dos imóveis vendidos nessa área (em MYR - Ringgit Malaio).
Um valor mediano significa que metade dos imóveis custam menos e metade custam mais.
Exemplo: 331800.0 MYR.

## Median_PSF (Price per Square Foot)

Preço mediano por pé quadrado (unidade de medida de área).
Indica o custo por área útil do imóvel.
Exemplo: 304.0 MYR por pé quadrado.

## Transactions

Número total de transações registradas para esse tipo de imóvel naquela área em 2025.
Exemplo: 593 indica um mercado mais ativo.

In [ ]:
missing_data = df.isnull().sum()

missing_data = missing_data[missing_data > 0]

if missing_data.empty:
    print("Não há dados faltantes no dataset.")
else:
    print("Dados faltantes encontrados:\n", missing_data)


In [ ]:
# Distribuição dos preços medianos
plt.figure(figsize=(10, 6))
plt.hist(df["Median_Price"].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title("Distribuição de Preços Medianos")
plt.xlabel("Preço Mediano (MYR)")
plt.ylabel("Número de Propriedades")
plt.grid(True)
plt.show()

In [ ]:
# Relação entre preço mediano e preço por pé quadrado
plt.figure(figsize=(10, 6))
plt.scatter(df["Median_PSF"], df["Median_Price"], alpha=0.5, color='purple')
plt.title("Preço Mediano vs. Preço por Pé Quadrado")
plt.xlabel("Preço por Pé Quadrado (MYR)")
plt.ylabel("Preço Mediano (MYR)")
plt.grid(True)
plt.show()

In [ ]:
# Relação entre transações e preço
plt.figure(figsize=(10, 6))
sns.lineplot(x="Transactions", y="Median_Price", data=df, marker="o")
plt.title("Preço Mediano por Número de Transações")
plt.xlabel("Número de Transações")
plt.ylabel("Preço Mediano (MYR)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(df["Median_Price"])

In [ ]:
# Método do IQR para remover outliers
Q1 = df["Median_Price"].quantile(0.25)
Q3 = df["Median_Price"].quantile(0.75)
IQR = Q3 - Q1

# Definir limites inferior e superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar os dados sem os outliers
df = df[(df["Median_Price"] >= limite_inferior) & (df["Median_Price"] <= limite_superior)]

# Criando um boxplot depois da remoção dos outliers
plt.figure(figsize=(8, 5))
sns.boxplot(df["Median_Price"])
plt.title("Boxplot após a remoção de outliers")
plt.show()

In [ ]:
y = df['Median_Price']

# Definindo as colunas categóricas
categorical_columns = ["Township", "Area", "State", "Tenure", "Type"]
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_cats = encoder.fit_transform(df[categorical_columns])

# Converter para DataFrame
encoded_cats_df = pd.DataFrame(
    encoded_cats, 
    columns=encoder.get_feature_names_out(categorical_columns)
)
encoded_cats_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Definindo as colunas numéricas
numeric_columns = ["Transactions"]

# Criar o DataFrame final (sem aplicar escalonamento ainda)
X = pd.concat([encoded_cats_df, df[numeric_columns].reset_index(drop=True)], axis=1)

print(X.head(),y.head())

O escalonamento é essencial para a regressão linear, pois o modelo é sensível à magnitude das variáveis, o que impacta diretamente os coeficientes e o desempenho. Em contrapartida, modelos de árvore, como Decision Tree, Random Forest e Gradient Boosting, não se beneficiam, já que realizam divisões baseadas nos valores dos dados, independentemente da escala. Por isso, utilizei um pipeline para aplicar o escalonamento apenas ao modelo que realmente exige essa transformação.

In [ ]:
# Definindo os modelos com pré-processamento ideal
models = {
    'Linear Regression': Pipeline([
        ('scaler', StandardScaler()),  # Escalonamento apenas para regressão linear
        ('model', LinearRegression())
    ]),
    'Decision Tree': Pipeline([
        ('model', DecisionTreeRegressor())
    ]),
    'Random Forest': Pipeline([
        ('model', RandomForestRegressor())
    ]),
    'Gradient Boosting': Pipeline([
        ('model', GradientBoostingRegressor())
    ])
}

# Definindo os parâmetros para o GridSearch
param_grids = {
    'Linear Regression': {},
    'Decision Tree': {
        'model__max_depth': [3, 5, 10],
        'model__min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [3, 5, 10]
    },
    'Gradient Boosting': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 10]
    }
}

# Definir o KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Avaliar os modelos com GridSearchCV e KFold Cross-Validation
for model_name, pipeline in models.items():
    print(f"Model: {model_name}")
    
    # Inicializar o GridSearchCV
    grid_search = GridSearchCV(
        estimator=pipeline, 
        param_grid=param_grids[model_name], 
        cv=kf, 
        scoring='neg_mean_squared_error'
    )
    
    # Variáveis para armazenar resultados
    mse_scores = []
    rmse_scores = []
    r2_scores = []

    # Loop KFold para treinamento e validação
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Treinar o modelo com GridSearchCV
        grid_search.fit(X_train, y_train)
        
        # Obter o melhor modelo
        best_model = grid_search.best_estimator_
        
        # Fazer previsões
        y_pred = best_model.predict(X_test)
        
        # Calcular as métricas
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # Armazenar as métricas
        mse_scores.append(mse)
        rmse_scores.append(rmse)
        r2_scores.append(r2)

    # Exibir os melhores parâmetros e as métricas
    # print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
    # print(f"Mean Mean Squared Error (MSE): {np.mean(mse_scores)}")
    # print(f"Mean Root Mean Squared Error (RMSE): {np.mean(rmse_scores)}")
    # print(f"Mean R²: {np.mean(r2_scores)}\n")

## Model: Linear Regression

**Best Parameters:** `{}`  

- **Mean Squared Error (MSE):** 31976962362.72  
- **Root Mean Squared Error (RMSE):** 178677.84  
- **R²:** 0.2079  

---

## Model: Decision Tree

**Best Parameters:** `{'model__max_depth': 10, 'model__min_samples_split': 10}`  

- **Mean Squared Error (MSE):** 29375160426.74  
- **Root Mean Squared Error (RMSE):** 171240.56  
- **R²:** 0.27075  

---

## Model: Random Forest

**Best Parameters:** `{'model__max_depth': 10, 'model__n_estimators': 200}`  

- **Mean Squared Error (MSE):** 26388190268.63  
- **Root Mean Squared Error (RMSE):** 162242.87  
- **R²:** 0.3454 

---

## Model: Gradient Boosting

**Best Parameters:** `{'model__learning_rate': 0.2, 'model__max_depth': 10, 'model__n_estimators': 200}`  

- **Mean Squared Error (MSE):** 24606648955.43  
- **Root Mean Squared Error (RMSE):** 156691.50  
- **R²:** 0.3897 
